<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [5]</a>'.</span>

In [1]:
from IPython import get_ipython
from IPython.core.display import display
get_ipython().run_line_magic('load_ext', 'autoreload')
get_ipython().run_line_magic('autoreload', '2')
get_ipython().run_line_magic('run', 'setup')

In [2]:
#leeftijdsgroepen = SimpleNamespace()
@run
def cell():
  global rivm
  rivm = RIVM.csv('COVID-19_casus_landelijk')
  display(rivm.head())

downloading rivm/COVID-19_casus_landelijk-2021-09-14@13-15.csv


rivm COVID-19_casus_landelijk zipping rivm/COVID-19_casus_landelijk-2021-09-14@13-15.csv


loading rivm/COVID-19_casus_landelijk-2021-09-14@13-15.csv.gz


,Date_file,Date_statistics,Date_statistics_type,Agegroup,Sex,Province,Hospital_admission,Deceased,Week_of_death,Municipal_health_service
0,2021-09-14 10:00:00,2020-01-01,DOO,40-49,Female,Noord-Holland,No,No,NaN,GGD Amsterdam
1,2021-09-14 10:00:00,2020-01-01,DOO,50-59,Male,Gelderland,No,No,NaN,Veiligheids- en Gezondheidsregio Gelderland-Mi...
2,2021-09-14 10:00:00,2020-01-01,DOO,20-29,Female,Zuid-Holland,No,No,NaN,GGD Hollands-Midden
3,2021-09-14 10:00:00,2020-01-01,DOO,60-69,Female,Noord-Holland,No,No,NaN,GGD Hollands-Noorden
4,2021-09-14 10:00:00,2020-01-04,DOO,10-19,Female,Gelderland,Unknown,No,NaN,GGD Gelderland-Zuid


In [3]:
@run
def cell():
  global bevolking
  bevolking = CBS.bevolking(leeftijdsgroepen=True)

In [4]:
# Bereken de stand van zaken van besmettingen / hospitalisaties / overlijden, per cohort in absolute aantallen en aantallen per 100k, met een kleur indicator voor de aantallen.
# vervang <50 en Unknown door Onbekend
@run
def cell():
  rivm['Cohort'] = rivm['Agegroup'].replace({'<50': 'Onbekend', 'Unknown': 'Onbekend'})
  # aangenomen 'gemiddelde' leeftijd van een cohort: minimum waarde + 5
  assumed_cohort_age = [(cohort, [int(n) for n in cohort.replace('+', '').split('-')]) for cohort in rivm['Cohort'].unique() if cohort[0].isdigit()]
  assumed_cohort_age = { cohort: min(rng) + 5 for cohort, rng in assumed_cohort_age }
  rivm['Gemiddelde leeftijd'] = rivm['Cohort'].apply(lambda x: assumed_cohort_age.get(x, np.nan))

  # verwijder tijd
  rivm['Date_file_date'] = pd.to_datetime(rivm['Date_file'].replace(r' .*', '', regex=True))

  rivm['Date_statistics_date'] = pd.to_datetime(rivm['Date_statistics'])

  # weken terug = verschil tussen Date_file en Date_statistcs, gedeeld door 7 dagen
  rivm['Weken terug'] = np.floor((rivm['Date_file_date'] - rivm['Date_statistics_date'])/np.timedelta64(7, 'D')).astype(int)

  # voeg key, gem leeftijd, kleurnummer en totaal toe
  Date_file = rivm['Date_file_date'].unique()[0].astype('M8[D]').astype('O')
  cohorten = list(bevolking.index) + ['Onbekend']
  def summarize(df, category, prefix):
    # aangezien we hier de dataframe in-place wijzigen (bijv door toevoegen kolommen)
    # en we het 'rivm' frame later nog clean nodig hebben
    df = df.copy(deep=True)

    df = (df
          .groupby(['Weken terug', 'Cohort'])['count']
          .sum()
          .unstack(fill_value=np.nan)
          .reset_index()
          .rename_axis(None, axis=1)
        ).merge(df
          # we voegen hier gemiddelde leeftijd toe, want die willen we op een ander
          # niveau aggregeren voor 'df' overschreven word
          .groupby(['Weken terug'])['Gemiddelde leeftijd']
          .mean()
          .to_frame(), on='Weken terug'
        )

    # altijd 52 rijen
    df = pd.Series(np.arange(52), name='Weken terug').to_frame().merge(df, how='left', on='Weken terug')

    # toevoegen missende cohorten
    for col in cohorten:
      if not col in df:
        df[col] = np.nan

    # sommeer per rij (axis=1) over de cohorten om een totaal te krijgen
    df['Totaal'] = df[cohorten].sum(axis=1)

    # voeg periode en datum toe
    # periode afgeleid van weken-terug (= de index voor deze dataframe)
    df['Datum'] = pd.to_datetime(Date_file)
    df['Periode'] = (df
      .index.to_series()
      .apply(
        lambda x: (
          (Date_file + datetime.timedelta(weeks=-(x+1), days=1)).strftime('%d/%m')
          + '-'
          + (Date_file + datetime.timedelta(weeks=-x)).strftime('%d/%m')
        )
      )
    )

    # voeg 'Key' en 'Type' kolom toe. Variabele 'type' kan niet, is een language primitive.
    df['Key'] = prefix + df.index.astype(str).str.rjust(3, fillchar='0')
    df['Type'] = category

    # voeg de kleur kolommen toe
    for col in cohorten:
      df['c' + col] = ((df[col] / df[[col for col in cohorten]].max(axis=1)) * 1000).fillna(0).astype(int)

    # herschikken van de kolommen
    colorder = ['Key', 'Weken terug', 'Datum', 'Periode', 'Gemiddelde leeftijd', 'Totaal', 'Type']
    return df[colorder + [col for col in df if col not in colorder]]

  factor = bevolking.to_dict()['per 100k']
  global tabel
  tabel = pd.concat(
    # flatten the result list zodat pd.concat ze onder elkaar kan plakken
    functools.reduce(lambda a, b: a + b, [
      [summarize(df.assign(count=1), label, prefix), summarize(df.assign(count=df['Cohort'].apply(lambda x: factor.get(x, np.nan))), label + ' per 100.000', prefix + '100k')]
      for df, label, prefix in [
        (rivm, 'Positief getest', 'p'), # volledige count per cohort
        (rivm[rivm.Hospital_admission == 'Yes'], 'Ziekenhuisopname', 'h'), # count van cohort voor Hospital_admission == 'Yes'
        (rivm[rivm.Deceased == 'Yes'], 'Overleden', 'd'), # count van cohort voor Deceased == 'Yes'
      ]
    ])
  )

  # rood -> groen
  cdict = {
    'red':   ((0.0, 0.0, 0.0),   # no red at 0
              (0.5, 1.0, 1.0),   # all channels set to 1.0 at 0.5 to create white
              (1.0, 0.8, 0.8)),  # set to 0.8 so its not too bright at 1
    'green': ((0.0, 0.8, 0.8),   # set to 0.8 so its not too bright at 0
              (0.5, 1.0, 1.0),   # all channels set to 1.0 at 0.5 to create white
              (1.0, 0.0, 0.0)),  # no green at 1
    'blue':  ((0.0, 0.0, 0.0),   # no blue at 0
              (0.5, 1.0, 1.0),   # all channels set to 1.0 at 0.5 to create white
              (1.0, 0.0, 0.0))   # no blue at 1
  }
  cm = colors.LinearSegmentedColormap('GnRd', cdict)
  # geel -> paars
  cm = sns.color_palette('viridis_r', as_cmap=True)
  display(tabel
    .fillna(0)
    .head()
    .round(1)
    .reset_index(drop=True)
    .style.background_gradient(cmap=cm, axis=1, subset=cohorten)
  )

,Key,Weken terug,Datum,Periode,Gemiddelde leeftijd,Totaal,Type,0-9,10-19,20-29,30-39,40-49,50-59,60-69,70-79,80-89,90+,Onbekend,c0-9,c10-19,c20-29,c30-39,c40-49,c50-59,c60-69,c70-79,c80-89,c90+,cOnbekend
0,p000,0,2021-09-14 00:00:00,08/09-14/09,30.800000,8556.000000,Positief getest,1399.000000,1812.000000,1433.000000,1263.000000,1080.000000,730.000000,425.000000,241.000000,137.000000,36.000000,0.000000,772,1000,790,697,596,402,234,133,75,19,0
1,p001,1,2021-09-14 00:00:00,01/09-07/09,31.500000,16245.000000,Positief getest,2268.000000,3383.000000,2919.000000,2633.000000,1990.000000,1420.000000,886.000000,445.000000,250.000000,51.000000,0.000000,670,1000,862,778,588,419,261,131,73,15,0
2,p002,2,2021-09-14 00:00:00,25/08-31/08,33.100000,16980.000000,Positief getest,1707.000000,3310.000000,3574.000000,2840.000000,2109.000000,1585.000000,1033.000000,487.000000,269.000000,66.000000,0.000000,477,926,1000,794,590,443,289,136,75,18,0
3,p003,3,2021-09-14 00:00:00,18/08-24/08,33.600000,17169.000000,Positief getest,1243.000000,3340.000000,4094.000000,2871.000000,2186.000000,1665.000000,938.000000,481.000000,277.000000,73.000000,1.000000,303,815,1000,701,533,406,229,117,67,17,0
4,p004,4,2021-09-14 00:00:00,11/08-17/08,33.200000,16763.000000,Positief getest,1019.000000,3352.000000,4564.000000,2667.000000,1866.000000,1628.000000,918.000000,419.000000,239.000000,91.000000,0.000000,223,734,1000,584,408,356,201,91,52,19,0


<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [5]:
if knack:
  await knack.publish(tabel.fillna(0).assign(Datum=tabel.Datum.dt.strftime('%Y-%m-%d')), 'Leeftijdsgroep', Cache)

infinities:


Empty DataFrame
Columns: []
Index: []


nan:


Empty DataFrame
Columns: []
Index: []


Key                     object
Weken terug              int64
Datum                   object
Periode                 object
Gemiddelde leeftijd    float64
Totaal                 float64
Type                    object
0-9                    float64
10-19                  float64
20-29                  float64
30-39                  float64
40-49                  float64
50-59                  float64
60-69                  float64
70-79                  float64
80-89                  float64
90+                    float64
Onbekend               float64
c0-9                     int64
c10-19                   int64
c20-29                   int64
c30-39                   int64
c40-49                   int64
c50-59                   int64
c60-69                   int64
c70-79                   int64
c80-89                   int64
c90+                     int64
cOnbekend                int64
dtype: object
updating knack


...

BatchName     object
ObjectName    object
Source        object
Timestamp     object
dtype: object


  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:01<00:00,  1.00s/it]

100%|██████████| 1/1 [00:01<00:00,  1.01s/it]

URLError: <urlopen error [Errno 104] Connection reset by peer>